<a href="https://colab.research.google.com/github/unicornlaunching/weaviate-tutorials/blob/main/ai_quickstart_weaviate_starting_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What if we could create a viral twitterbot using AI & Weaviate?

#### Click on the picture below to watch the tutorial.

---

[![Watch the video](https://github.com/unicornlaunching/weaviate-tutorials/blob/main/youtubethumbnailfortutorial.jpeg?raw=true)](https://www.youtube.com/watch?v=HRLVDHU_FVA)

# Step 1: Create a Weaviate Account and create a Weaviate Cloud Cluster

You can start by visiting https://console.weaviate.cloud/ and create a Weaviate Cluster to house your data.

![Weaviate Cluster Creation](https://github.com/unicornlaunching/weaviate-tutorials/blob/main/Screen%20Shot%202023-10-15%20at%204.24.22%20PM.png?raw=true)

I got this [tutorial](https://weaviate.io/developers/weaviate/quickstart) from the docs on weaviate's site.

In [ ]:
!pip install weaviate-client

In [ ]:
import weaviate
import json

client = weaviate.Client(
    url = "INSERT_WEAVIATE_ENDPOINT_HERE",  # Replace with your endpoint created in https://console.weaviate.cloud/dashboard
    auth_client_secret=weaviate.AuthApiKey(api_key="INSERT_WEAVIATE_API_KEY_HERE"),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": "INSERT_OPENAI_API_KEY_HERE"  # Replace with your OPENAI API key here
    }
)

---

In [ ]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

---

In [ ]:
import requests
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

client.batch.configure(batch_size=100)  # Configure batch
with client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(data):  # Batch import data
        print(f"importing question: {i+1}")
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

---

In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["biology"]})
    .with_generate(single_prompt="Explain {answer} as you might to a five-year-old.")
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

---

In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["science"]})
    .with_generate(single_prompt="Explain {answer} as you might to a five-year-old.")
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "Question": [
                {
                    "_additional": {
                        "generate": {
                            "error": null,
                            "singleResult": "Imagine you are running really fast, and the wind is blowing against your face. You can feel the wind pushing you back, right? Now, imagine you are running even faster, so fast that you are almost as fast as a speeding car. When you reach that super-fast speed, something interesting happens. The wind around you gets so strong that it becomes really hard for you to move forward. It's like a big wall of wind that stops you from going any faster. That wall of wind is called the sound barrier.\n\nYou see, sound is like tiny vibrations that travel through the air. When you make a sound, like clapping your hands, those vibrations move through the air and reach our ears, so we can hear it. But when something moves really, really fast, like an airplane or a 

---

In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text({"concepts": ["science"]})
    .with_generate(grouped_task="Write a tweet with emojis about these facts.")
    .with_limit(2)
    .do()
)

print(response["data"]["Get"]["Question"][0]["_additional"]["generate"]["groupedResult"])

"🛩️💨 Breaking the sound barrier! ✈️💥 Did you know that in 70-degree air, a plane traveling at about 1,130 feet per second can break it? 🌬️🔊 #ScienceFacts"
